# Tratando Missing Values.

Veremos várias maneiras de se lidar com esses dados faltantes, são tantas opções
e seus usos variam de acordo com as situações.

O link que usei como apoio foi esse:https://www.youtube.com/watch?v=Xdqavu26oJ8&list=WL&index=14&t=724s

## Sumário
    1 - Identificando  missing values
    2 - O que fazer?
        2.1 Excluir
        2.2 Substituir
        2.3 Outros parâmetros de substituição
        2.4 fillna(forward and backward)
    3 - Missing values não reconhecidos

In [121]:
# Seção de imports

# Manipulação dos dados
import pandas as pd

# Função específica ao final (np.nan)
import numpy as np

In [12]:
# Receber os dados na memória

df = pd.read_csv('Diabetes.csv')

In [13]:
# Como de costume uma olhada na tabela 
df.head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1.0
1,1,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31,NaN
2,2,8.0,183.0,64.0,NaN,NaN,23.3,0.672,32,1.0
3,3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,NaN
4,4,NaN,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0


### Identificando missing values
    Durante estes passos da casual análise exploratória, já seria
    possível saber que existem valores nulos/faltantes/na/NaN/Null.

In [14]:
df.shape

(768, 10)

In [16]:
'''
    Em .shape sabemos que há 768 linhas na tabela, logo ao ver abaixo que 
    tem colunas com menos de 768 valores não nulos, percebemos que há
    NaN nos dados.
'''

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                768 non-null    int64  
 1   Pregnancies               657 non-null    float64
 2   Glucose                   763 non-null    float64
 3   BloodPressure             733 non-null    float64
 4   SkinThickness             541 non-null    float64
 5   Insulin                   394 non-null    float64
 6   BMI                       757 non-null    float64
 7   DiabetesPedigreeFunction  768 non-null    float64
 8   Age                       768 non-null    int64  
 9   Outcome                   268 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


    Essa é uma maneira de se perceber. 
    Mas há algumas funções específicas que 
    podem nos auxiliar nesse trabalho.

In [18]:
'''
    is.null() nos mostra os valores nulos existentes. Porém, com um
    dataset desse tamanho ou maiores, fica difícil de se avaliar todos 
    um por um, por isso...
'''

df.isnull()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,True,False,False,False,True
2,False,False,False,False,True,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True
4,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,False,False,False,True
764,False,False,False,False,False,True,False,False,False,True
765,False,False,False,False,False,False,False,False,False,True
766,False,False,False,False,True,True,False,False,False,False


In [31]:
'''
    Utilizar o "buscador de nulls" com .sum() é uma boa ideia
    para saber quantos NaN estão escondidos pela tabela e em
    quais colunas eles se encontram
'''
# Obs: isna() faz o mesmo que isnull()

df.isna().sum()

Unnamed: 0                    0
Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [32]:
# Desse modo temos o total de valores NaN

df.isna().sum().sum()

1263

### 2 - O que fazer?
    Ao lidar com missing values temos basicamente 2 opções:
    Substituí-los ou Removê-los.
    
    Depende do contexto, fazer uma análise sobre um tema com 3 pessoas em
    um local onde haviam 1000 envolvidos nos torna vuneráveis a errar por não
    ter explorado nossos dados o suficiente. Ou seja, se o número de NaN
    for muito alto ao ponto de invalidar a análise, podemos simplesmente 
    excluir todos esse dados. 
    Mas caso seja possível torná-los em um valor útil, podemos substituí-los,
    por exemplo, pela média ou mediana (normalmente não indicado), ou se for uma
    categoria poderíamos talvez trocar o valor Null por 'indisponível'. Você terá 
    que decidir de acordo com seus objetivos.    

#### 2.1 Excluir

In [28]:
df.head(1)

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1.0


In [36]:
# Caso queria excluir, podemos fazer isso de vários modos, dentre eles:
'''
    Como você pode ver a tabela agora começa pelo índice 6 (e já pula para o 8).
    Isso ocorre porque por padrão .dropna() elimina as linhas em que haja QUALQUER
    dados Null.
    Em df.head(1) notamos que apenas a coluna ['Insulin'] tem valor nulo, mas como
    essa linha tem pelo menos 1 valor nulo, ela será excluida.
'''

df.dropna()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,6,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26,1.0
8,8,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53,1.0
13,13,1.0,189.0,60.0,23.0,846.0,30.1,0.398,59,1.0
14,14,5.0,166.0,72.0,19.0,175.0,25.8,0.587,51,1.0
19,19,1.0,115.0,70.0,30.0,96.0,34.6,0.529,32,1.0
...,...,...,...,...,...,...,...,...,...,...
730,730,3.0,130.0,78.0,23.0,79.0,28.4,0.323,34,1.0
732,732,2.0,174.0,88.0,37.0,120.0,44.5,0.646,24,1.0
740,740,11.0,120.0,80.0,37.0,150.0,42.3,0.785,48,1.0
748,748,3.0,187.0,70.0,22.0,200.0,36.4,0.408,36,1.0


In [35]:
'''
    Ao mudarmos o parâmetro "how", do padrão (how='any') para how='all', a função
    apenas deletará as linhas cujos TODOS os valores são NaN. Logo, dessa vez a linha
    de índice 0 não será deletada
'''

df.dropna(how='all')

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1.0
1,1,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31,NaN
2,2,8.0,183.0,64.0,NaN,NaN,23.3,0.672,32,1.0
3,3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,NaN
4,4,NaN,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0
...,...,...,...,...,...,...,...,...,...,...
763,763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63,NaN
764,764,2.0,122.0,70.0,27.0,NaN,36.8,0.340,27,NaN
765,765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30,NaN
766,766,1.0,126.0,60.0,NaN,NaN,30.1,0.349,47,1.0


### Inplace 
    Talvez você tenha percebido, mas o .dropna() não realmente alterou a 
    tabela. A exclusão foi feita em uma cópia, o que vimos foi uma representação
    de como a tabela ficaria caso tirassemos aqueles dados.
    
    Para realmente modificarmos a variável 'df' temos que habilitar o
    parâmetro 'inplace' como inplace=True
    df.dropna(inplace=True)
    
    Ou então atribuir a tabela-cópia à uma nova variável
    df2 = df.dropna()

#### 2.2 Substituir

In [47]:
# Se a opção seja substituir, usamos

df.fillna('SUBSTITUÍDO').head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6,148,72,35,SUBSTITUÍDO,33.6,0.627,50,1
1,1,1,85,66,29,SUBSTITUÍDO,26.6,0.351,31,SUBSTITUÍDO
2,2,8,183,64,SUBSTITUÍDO,SUBSTITUÍDO,23.3,0.672,32,1
3,3,1,89,66,23,94,28.1,0.167,21,SUBSTITUÍDO
4,4,SUBSTITUÍDO,137,40,35,168,43.1,2.288,33,1


In [48]:
# Ou por 0

df.fillna(0).head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50,1.0
1,1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31,0.0
2,2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32,1.0
3,3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,0.0
4,4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0


#### 2.3 Outros parâmetros de substituição

In [73]:
# Ou na coluna específica substituir pela média
# mean significa média
mean = df['Insulin'].mean()

df['Insulin'].fillna(mean)

0      155.548223
1      155.548223
2      155.548223
3       94.000000
4      168.000000
          ...    
763    180.000000
764    155.548223
765    112.000000
766    155.548223
767    155.548223
Name: Insulin, Length: 768, dtype: float64

Obs: A regra do inplace também vale aqui.

Caso você queira preencher várias colunas com medidas diferentes,
você utilizar um dicionário para tal:

In [83]:
teste_A_B_C = {'Insulin': 'AAAA','SkinThickness':'BBBB','Pregnancies':'CCCC'}

df.fillna(teste_A_B_C).head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,6,148.0,72.0,35,AAAA,33.6,0.627,50,1.0
1,1.0,1,85.0,66.0,29,AAAA,26.6,0.351,31,NaN
2,2.0,8,183.0,64.0,BBBB,AAAA,23.3,0.672,32,1.0
3,3.0,1,89.0,66.0,23,94,28.1,0.167,21,NaN
4,4.0,CCCC,137.0,40.0,35,168,43.1,2.288,33,1.0


In [84]:
'''
    Em um exemplo um pouco mais real, passei a média, moda e mediana para
    algumas variáveis e as utilizei na substituição
'''

insulin_mean = df['Insulin'].mean()
skinThickness_mode = df['SkinThickness'].mode()
pregnancies_median = df['Pregnancies'].median()

print('insulin_mean = ',insulin_mean)
print('skinThickness_mode = ',skinThickness_mode)
print('pregnancies_median = ',pregnancies_median)

valores_de_preenchimento = {'Insulin': insulin_mean,'SkinThickness':skinThickness_mode,'Pregnancies':pregnancies_median}

df.fillna(valores_de_preenchimento).head()

insulin_mean =  155.5482233502538
skinThickness_mode =  0    32.0
dtype: float64
pregnancies_median =  4.0


,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,6.0,148.0,72.0,35.0,155.548223,33.6,0.627,50,1.0
1,1.0,1.0,85.0,66.0,29.0,155.548223,26.6,0.351,31,NaN
2,2.0,8.0,183.0,64.0,NaN,155.548223,23.3,0.672,32,1.0
3,3.0,1.0,89.0,66.0,23.0,94.000000,28.1,0.167,21,NaN
4,4.0,4.0,137.0,40.0,35.0,168.000000,43.1,2.288,33,1.0


#### 2.4 fillna(forward and backward) 
    Forward = para frente
    Backward = para trás
    
    Um  preenche os valores "de cima para baixo"
    e o outro "de baixo para cima", com os valores já existentes.
    Exemplo:
             Forward     Backward
    20          20           20
    NaN         20           30
    30          30           30   

In [89]:
# A tabela original

df.head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1.0
1,1.0,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31,NaN
2,2.0,8.0,183.0,64.0,NaN,NaN,23.3,0.672,32,1.0
3,3.0,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,NaN
4,4.0,NaN,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0


In [94]:
# Usamos o parâmetro method='ffill' para Forward  ----  "de cima para baixo"

'''A coluna ['Insulim'] ainda tem NaN, já que não há valores no início para se utilizar'''

df.fillna(method='ffill').head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1.0
1,1.0,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31,1.0
2,2.0,8.0,183.0,64.0,29.0,NaN,23.3,0.672,32,1.0
3,3.0,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,1.0
4,4.0,1.0,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0


In [92]:
# E method='bfill' para Backward  ----  "de baixo para cima"

df.fillna(method='bfill').head()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.0,6.0,148.0,72.0,35.0,94.0,33.6,0.627,50,1.0
1,1.0,1.0,85.0,66.0,29.0,94.0,26.6,0.351,31,1.0
2,2.0,8.0,183.0,64.0,23.0,94.0,23.3,0.672,32,1.0
3,3.0,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,1.0
4,4.0,5.0,137.0,40.0,35.0,168.0,43.1,2.288,33,1.0


### 3 - Missing values não reconhecidos
    Valores faltantes, na verdade, não são valores que faltam literalmente,
    mas são valores que não estão de acordo com a 'categoria' daqueles dados.
    Para ser mais preciso, que não estão de acordo com a categoria e modo de
    nosso uso.
    
    Observe este novo DataFrame criado a partir de um dicionário:

In [115]:
people_data = {
    'first': ['12', 'Jane', 'John', 'Chris', np.nan, None, 'NA'], 
    'last': ['Schafer', 'Doe', 'Doe', 'Schafer', np.nan, np.nan, 'Missing'], 
    'email': ['CoreyMSchafer@gmail.com', 'JaneDoe@email.com', 'JohnDoe@email.com', None, np.nan, 
              'Anonymous@email.com', 'NA'],
    'age': ['33', '55', '63', '36', None, None, 'Missing']
}

data_frame = pd.DataFrame(people_data)

data_frame

,first,last,email,age
0,12,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


    Vemos que há valores nulos, porém há valores que não têm uso para nós
    mas que são considerados não nulos pela linguagem.    
    Observe bem e Compare as tabelas

In [99]:
data_frame.isna()

,first,last,email,age
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,True,True,True,True
5,True,True,False,True
6,False,False,False,False


In [117]:
# Atenção nessa soma, voltaremos a ela logo logo
data_frame.isna().sum()

first    2
last     2
email    2
age      2
dtype: int64

    Os valores NA, Missing (na última linha) e 12 (na primeira) não são considerados nulos
    Nesses casos, uma das ações que podemos tomar é criar uma lista com esses valores e 
    1 - Fazer com que a linguagem olhe para eles como NaN na hora de importar os dados. Ou
    2 - Simplesmente substituí-los pelo valor NaN (usando a função np.nan da biblioteca numpy)

In [114]:
'''    1    '''
# Criando lista de valores que sabemos ser NaN
lista_de_NaN = ["NA", "Missing", 12]

# Importar passando lista_de_NaN como valores NaN       index_col=0 --> para não criar novo index
data_frame_exemple = pd.read_csv('ultimo_exemplo.csv', na_values = lista_de_NaN, index_col=0)
data_frame_exemple

,first,last,email,age
0,NaN,Schafer,CoreyMSchafer@gmail.com,33.0
1,Jane,Doe,JaneDoe@email.com,55.0
2,John,Doe,JohnDoe@email.com,63.0
3,Chris,Schafer,NaN,36.0
4,NaN,NaN,NaN,NaN
5,NaN,NaN,Anonymous@email.com,NaN
6,NaN,NaN,NaN,NaN


In [118]:
'''    2    '''
# Criando lista de NaN    
lista_de_NaN = ["NA", "Missing", '12']

# np.nan retorna um valor nulo (NaN)
data_frame.replace(lista_de_NaN, np.nan, inplace=True)
data_frame

,first,last,email,age
0,NaN,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NaN,NaN,NaN,NaN


In [120]:
# Agora o Pandas consegue identificar os valores como NaN
data_frame.isna().sum()

first    4
last     3
email    3
age      3
dtype: int64

    Por hoje é isso, há muito mais a se aprender, acredito que esse
    resumo ficou bem completo para quem está no começo, mas 
    continue pequisando, aprendendo e se desenvolvendo sempre.